In [ ]:
!pip install transformers[torch] pandas datasets pyarrow scikit-learn



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s eta 0:00:00


In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/nlp/train.csv')
df.head()

,index,sentence,slots,intent_label
0,0,کارت به کارت میخوام از کارت ملی به شماره کارت ...,B-Method I-Method I-Method O O O B-Bankname O ...,transaction_card
1,1,میخوام 50 هزار تومن به علی باقری کارت به کارت کنم,O B-Amount I-Amount I-Amount O B-Username I-Us...,transaction_card
2,2,میخواهم پولی را از حساب ملت با شماره 48759214 ...,O O O O O B-Bankname O O B-Accountnumber O O B...,transaction_card
3,3,میخوام به حساب علی احمدی از حساب ملتم کارت به ...,O O O B-Username I-Username O O B-Bankname B-M...,transaction_card
4,4,میخوام به حساب محمد توکلی پول کارت به کارت کنم,O O O B-Username I-Username O B-Method I-Metho...,transaction_card


In [ ]:
!pip install hazm

In [ ]:
from hazm import Normalizer
normalizerEn = Normalizer(persian_numbers=False)
text = "!!!!!!!!!!!123مشکل    این    جیه"
normalizer = Normalizer()
normalized_text = normalizer.normalize(text)
normalized_text


'!!!!!!!!!!! ۱۲۳ مشکل این جیه'

In [ ]:
from transformers.trainer import tqdm
from googletrans import Translator
import pandas as pd
from tqdm import tqdm
texts=[]
labels=[]

def translate_textEn(text):
    translator = Translator(service_urls=['translate.google.com'])
    translated = translator.translate(text, src='fa', dest='en')
    return translated.text

def translate_textFn(text):
    translator = Translator(service_urls=['translate.google.com'])
    translated = translator.translate(text, src='en', dest='fa')
    return translated.text

for i in range(2):
    for i in tqdm(range(len(df))):
        normal_text=normalizer.normalize(df['sentence'].iloc[i])
        texts.append( normal_text)
        labels.append(df['intent_label'].iloc[i])


        text=normalizerEn.normalize(df['sentence'].iloc[i])
        enText=translate_textEn(text)
        fnText=translate_textFn(enText)
        normal_text=normalizer.normalize(fnText)
        texts.append(normal_text)
        labels.append(df['intent_label'].iloc[i])

df = pd.DataFrame({'sentence': texts, 'intent_label': labels})

# ذخیره دیتافریم در فایل CSV
df.to_csv('data.csv', sep=',', index=False)

100%|██████████| 113/113 [00:17<00:00,  6.34it/s]


In [ ]:
len(df)

452

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    'HooshvareLab/bert-base-parsbert-uncased')


In [ ]:
#tokens = tokenizer.tokenize(df['sentence'].iloc[1])
normalizer = Normalizer()
normalized_text = normalizer.normalize(' شماره شبا IR432')
normalized_text
tokens = tokenizer.tokenize(normalized_text)

# چاپ توکن‌ها
print(tokens)


['شماره', 'شبا', 'ir', '۴۳۲']


In [ ]:
def process_data(row):

    text = row['sentence']
    text = str(text)
    text = ' '.join(text.split())

    encodings = tokenizer(text, padding="max_length",
                          truncation=True, max_length=25)

    label = 0
    if row['intent_label'] == 'transaction_card':
        label = 1
    elif row['intent_label'] == 'transaction_paya':
        label = 2
    elif row['intent_label'] == 'bill_payment':
        label = 3
    elif row['intent_label'] == 'balance_query':
        label = 4
    elif row['intent_label'] == 'introduce':
        label = 5
    encodings['label'] = label
    encodings['text'] = text

    return encodings

In [ ]:
processed_data = []

for i in range(len(df)):
    processed_data.append(process_data(df.iloc[i]))

len(processed_data)

452

In [ ]:
from sklearn.model_selection import train_test_split

new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)

In [ ]:
import pyarrow as pa
from datasets import Dataset

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'HooshvareLab/bert-base-parsbert-uncased',
    num_labels=6
)

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.071239
2,No log,0.005069
3,No log,0.003291


TrainOutput(global_step=138, training_loss=0.259468548539756, metrics={'train_runtime': 633.6517, 'train_samples_per_second': 1.709, 'train_steps_per_second': 0.218, 'total_flos': 13914038414700.0, 'train_loss': 0.259468548539756, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.003290837863460183,
 'eval_runtime': 8.1458,
 'eval_samples_per_second': 11.171,
 'eval_steps_per_second': 1.473,
 'epoch': 3.0}

In [ ]:
model.save_pretrained('./model/')

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model = AutoModelForSequenceClassification.from_pretrained('./model/').to(device)

In [ ]:
from transformers import AutoTokenizer

new_tokenizer = AutoTokenizer.from_pretrained('HooshvareLab/bert-base-parsbert-uncased')

In [ ]:
import torch
import numpy as np

def get_prediction(text):
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = new_model(**encoding)

    logits = outputs.logits
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    sigmoid = torch.nn.Sigmoid()
    print(sigmoid)
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1)

    if label == 1:
        return {
            'intent_label': 'transaction_card',
            'probability': probs[1]
        }
    elif label == 2:
        return {
            'intent_label': 'transaction_paya',
            'probability': probs[0]
        }
    elif label == 3:
        return {
            'intent_label': 'bill_payment',
            'probability': probs[0]
        }
    elif label == 4:
        return {
            'intent_label': 'balance_query',
            'probability': probs[0]
        }
    elif label == 5:
        return {
            'intent_label': 'introduce',
            'probability': probs[0]
        }
    # if row['intent_label'] == 'transaction_card':
    #     label = 1
    # elif row['intent_label'] == 'transaction_paya':
    #     label = 2
    # elif row['intent_label'] == 'bill_payment':
    #     label = 3
    # elif row['intent_label'] == 'balance_query':
    #     label = 4
    # elif row['intent_label'] == 'introduce':
    #     label = 5

In [ ]:
get_prediction('9,250 هزار تومن کارت به کارت کن به کارت محمود ریاحی,')

Sigmoid()


{'intent_label': 'transaction_card', 'probability': 0.99797255}